In [2]:
# import relevant libraries, datasets of interest, classifiers, and performance metrics
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.preprocessing import scale
digits = datasets.load_digits();

Each digit is comprised of an 8x8 pixel image.
We can reshape the image data to make a matrix in which each row represents an image, and the columns are the pixel values. 

In [3]:
nDigits = digits.images.shape[0]
# x = digits.images.reshape( nDigits, 64)
x = digits.data
x = scale(x)
x = np.insert( x, 0, 1, axis=1)
y = digits.target.reshape( nDigits, 1)

print 'There are %i total images of digits, each with 64 pixels.' % nDigits
print 'x has dimensions %s' %(x.shape,)
print 'y has dimensions %s' %(y.shape,)

There are 1797 total images of digits, each with 64 pixels.
x has dimensions (1797, 65)
y has dimensions (1797, 1)


We will perform logistic regression on the features contained in the pixel values of the digits. Lets define the cost function and its gradient.

In [4]:
from scipy.special import expit # the logistic function 

def CostFunction( theta, x, y, l):
    '''
    INPUTS:
    theta = parameters of logistic regression
    x = features of our data
    y = classifiers of our data
    l = "lambda," regularization parameter
    OUTPUTS:
    J = logistic regression cost function
    '''
    m = y.shape[0]
    h = expit(x.dot(theta))
    J =  np.log(h).dot(-y.T) - np.log( 1-h).dot( 1-y.T)
    J += theta.T.dot(theta) * l * 0.5 
    J = J / m;
    return J

def CostGradient( theta, x, y, l):
    '''
    INPUTS:
    theta = parameters of logistic regression
    x = features of our data
    y = classifiers of our data
    l = "lambda," regularization parameter
    OUTPUTS:
    gradJ = gradient of cost function
    '''
    m = y.shape[0];
    h = expit(x.dot(theta));
    gradJ = x.T.dot( expit(x.dot(theta)) - y.T)
    gradJ[1:] += theta[1:]*(1.*l)
    gradJ = gradJ*(1./m)
    return gradJ

In [5]:
# test to see if functions work without any errors
initial_theta = np.zeros((x.shape[1],1))
J = CostFunction( initial_theta, x, y, 0.0)
gradJ = CostGradient( initial_theta, x, y, 0.0)

We will use one vs all classification to train the model. Each class (in this case the digit value) will be trained independently of all other classes.

In [13]:
from scipy.optimize import fmin_cg

def ThetaOneVsAll( Nclass, x, y):
    '''
    Calculate the optimal parameter values of theta for each class
    INPUT:
    Nclass = number of classes
    OUTPUT
    thetaMatrix = Matrix of optimal parameters, each row corresponds to a class
    '''
    thetaMatrix = np.zeros((Nclass, len(x[0])))
    thetai = np.zeros((len(x[0]),1)) + 0.001
    thetai = thetai.reshape(len(thetai),)
    
    ytemp = y.reshape( len(y),)
    # one-vs-all classification
    # perform logistic regression for each class, training each independently
    for i in xrange(Nclass):
        yClass = np.array([ 1 if yi == 1 else 0 for yi in ytemp])
        print 'Optimizing for digit %d '%i
        
        out = fmin_cg( CostFunction, x0=thetai, fprime=CostGradient, \
                    args=(x,yClass,0.0), maxiter=50, disp=False)

        thetaMatrix[i,:] = out[0]

    return thetaMatrix

In [14]:
thetaMatrix = ThetaOneVsAll( 10, x, y)

Optimizing for digit 0 
Optimizing for digit 1 
Optimizing for digit 2 
Optimizing for digit 3 
Optimizing for digit 4 
Optimizing for digit 5 
Optimizing for digit 6 
Optimizing for digit 7 
Optimizing for digit 8 
Optimizing for digit 9 


The prediction for the i'th row is found to be the index at which Theta*X[i] is maximized.

In [15]:
def PredictOneVsAll( thetaMatrix, x):
    '''
    INPUTS:
    thetaMatrix = Matrix of optimal parameters, each row corresponds to a class
    OUTPUT:
    pred = predicted class for each corresponding to each row in X
    '''
    pred = np.argmax( expit(x.dot(thetaMatrix.T)), axis=1)
    return pred

def h(mytheta,myX): #Logistic hypothesis function
    return expit(np.dot(myX,mytheta))

def predict2(myTheta,myrow):
    """
    Function that computes a hypothesis for an individual image (row in X)
    and returns the predicted integer corresponding to the handwritten image
    """
    classes = range(10)
    hypots  = [0]*len(classes)
    #Compute a hypothesis for each possible outcome
    #Choose the maximum hypothesis to find result
    for i in xrange(len(classes)):
        hypots[i] = h(myTheta[i],myrow)
    return classes[np.argmax(np.array(hypots))]    

In [28]:
yPredict = PredictOneVsAll( thetaMatrix, x)
# print yPredict[:20]
# print y[:20].T

for irow in xrange(x.shape[0]):
    yPredict[irow] = predict2(thetaMatrix,x[irow])
    
print yPredict[:10]
print y[:10].T
yPredict[1] == y[1][0]

[0 0 0 0 0 0 0 0 0 0]
[[0 1 2 3 4 5 6 7 8 9]]


1797

In [29]:
def predictOneVsAll(myTheta,myrow):
    """
    Function that computes a hypothesis for an individual image (row in X)
    and returns the predicted integer corresponding to the handwritten image
    """
    classes = [10] + range(1,10)
    hypots  = [0]*len(classes)
    #Compute a hypothesis for each possible outcome
    #Choose the maximum hypothesis to find result
    for i in xrange(len(classes)):
        hypots[i] = expit(myrow.dot(myTheta[i].T))
    return classes[np.argmax(np.array(hypots))]    

n_correct, n_total = 0., 0.
incorrect_indices = []
for i in xrange(x.shape[0]):
    n_total += 1
    if yPredict[i] == y[i][0]: 
        n_correct += 1
    else: incorrect_indices.append(irow)
print "Training set accuracy: %0.1f%%"%(100*(n_correct/n_total))

Training set accuracy: 9.9%
